In [ ]:
# Warning control
import os
import yaml
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
openai_api_key = os.environ.get('OPENAI_API_KEY')
if not openai_api_key:
  raise ValueError("Please set OPENAI_API_KEY environment variable")

from crewai import Agent, Task, Crew

In [5]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

In [6]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/content_creation_agents.yaml',
    'tasks': 'config/content_creation_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [9]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1'
groq_llm = "groq/llama-3.3-70b-versatile"

In [10]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

In [11]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Warehouse management and logistics innovation'
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 74685073-4c9e-4f93-a9dc-2424e79a4637                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: fd89735d-febc-4160-8af9-dd9c820f3063
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: fd89735d-febc-4160-8af9-dd9c820f3063
       Status: Executing Task...
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Warehouse management and logistics innovation. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Warehouse management and logistics innovation.



🤖 Agent: Lead Market Analyst

    Status: In Progress
└── 🧠 Thinking...

...

 Error during LLM call: litellm.BadRequestError: GroqException - Missing API Key - A call is being made to LLM Provider but no key is set either in the environment variables ({LLM_PROVIDER}_API_KEY) or via params
 An unknown error occurred. Please check the details below.
 Error details: litellm.BadRequestError: GroqException - Missing API Key - A call is being made to LLM Provider but no key is set either in the environment variables ({LLM_PROVIDER}_API_KEY) or via params


🚀 Crew: crew
└── 📋 Task: fd89735d-febc-4160-8af9-dd9c820f3063
       Assigned to: Lead Market Analyst
    
       Status: ❌ Failed
    └── 🤖 Agent: Lead Market Analyst
        
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: fd89735d-febc-4160-8af9-dd9c820f3063                                                                     │
│  Agent: Lead Market Analyst                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 74685073-4c9e-4f93-a9dc-2424e79a4637                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: GroqException - Missing API Key - A call is being made to LLM Provider but no key is set either in the environment variables ({LLM_PROVIDER}_API_KEY) or via params

In [12]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

NameError: name 'result' is not defined

In [13]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

NameError: name 'result' is not defined